In [1]:
import snowflake.connector
import pandas as pd
import numpy as np
from faker import Faker

In [20]:
import json

In [2]:
# Create a Faker instance
fake = Faker()

In [24]:
# Load credentials from the config.json file located in the "files" folder
with open('files/config.json') as config_file:
    config = json.load(config_file)

In [27]:
# Connect to Snowflake
conn = snowflake.connector.connect(
    user=config['SNOWFLAKE_USER'],
    password=config['SNOWFLAKE_PASSWORD'],
    account=config['SNOWFLAKE_ACCOUNT'],
    warehouse=config['SNOWFLAKE_WAREHOUSE'],
    role=config['SNOWFLAKE_ROLE']
)

In [28]:
# Create a cursor object
cursor = conn.cursor()

In [9]:
# Step 1: Use the existing Database
cursor.execute("USE DATABASE ANALYTICS_DB")

# Step 2: Use the Schema
cursor.execute("USE SCHEMA EMPLOYEE_DEPARTMENT_SCHEMA")

In [10]:
# Step 3: Create Tables in Snowflake

# Create Employees table
cursor.execute("""
    CREATE OR REPLACE TABLE EMPLOYEE_DEPARTMENT_SCHEMA.Employees (
        EmployeeID INT,
        EmployeeName STRING,
        DepartmentID INT
    );
""")

# Create Departments table
cursor.execute("""
    CREATE OR REPLACE TABLE EMPLOYEE_DEPARTMENT_SCHEMA.Departments (
        DepartmentID INT,
        DepartmentName STRING
    );
""")


In [11]:
# Generate and insert data using Faker
num_rows = 100000

# Generate data for Employees
employees = pd.DataFrame({
    'EmployeeID': np.arange(1, num_rows + 1),
    'EmployeeName': [fake.name() for _ in range(num_rows)],
    'DepartmentID': np.random.randint(1, 201, num_rows)  # Random department IDs from 1 to 200
})

In [12]:
# Generate data for Departments
departments = pd.DataFrame({
    'DepartmentID': np.arange(1, 201),
    'DepartmentName': [fake.company_suffix() for _ in range(200)]
})

In [13]:
# Convert data to tuples (for Snowflake insertion)
employees_records = [tuple(x) for x in employees.to_numpy()]
departments_records = [tuple(x) for x in departments.to_numpy()]

# Insert data into Employees table
insert_query = "INSERT INTO EMPLOYEE_DEPARTMENT_SCHEMA.Employees (EmployeeID, EmployeeName, DepartmentID) VALUES (%s, %s, %s)"
cursor.executemany(insert_query, employees_records)

# Insert data into Departments table
insert_query = "INSERT INTO EMPLOYEE_DEPARTMENT_SCHEMA.Departments (DepartmentID, DepartmentName) VALUES (%s, %s)"
cursor.executemany(insert_query, departments_records)

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Tables created successfully in ANALYTICS_DB, and data inserted.")

Tables created successfully in ANALYTICS_DB, and data inserted.


In [14]:
import time
import snowflake.connector

In [30]:
# Connect to Snowflake
conn = snowflake.connector.connect(
    user=config['SNOWFLAKE_USER'],
    password=config['SNOWFLAKE_PASSWORD'],
    account=config['SNOWFLAKE_ACCOUNT'],
    warehouse=config['SNOWFLAKE_WAREHOUSE'],
    role=config['SNOWFLAKE_ROLE']
)

# Create a cursor object
cursor = conn.cursor()


In [18]:
# Ensure database and schema are set
cursor.execute("USE DATABASE ANALYTICS_DB")
cursor.execute("USE SCHEMA EMPLOYEE_DEPARTMENT_SCHEMA")

In [19]:
# IN query
in_query = """
SELECT EmployeeName
FROM EMPLOYEE_DEPARTMENT_SCHEMA.Employees
WHERE DepartmentID IN (SELECT DepartmentID FROM EMPLOYEE_DEPARTMENT_SCHEMA.Departments WHERE DepartmentName = 'Sales');
"""

# EXISTS query
exists_query = """
SELECT EmployeeName
FROM EMPLOYEE_DEPARTMENT_SCHEMA.Employees e
WHERE EXISTS (SELECT 1 FROM EMPLOYEE_DEPARTMENT_SCHEMA.Departments d WHERE e.DepartmentID = d.DepartmentID AND d.DepartmentName = 'Sales');
"""

# Timing the IN query
start_time_in = time.time()
cursor.execute(in_query)
cursor.fetchall()  # Fetching results to ensure the query runs completely
end_time_in = time.time()

# Timing the EXISTS query
start_time_exists = time.time()
cursor.execute(exists_query)
cursor.fetchall()  # Fetching results to ensure the query runs completely
end_time_exists = time.time()

# Calculate execution times
in_query_time = end_time_in - start_time_in
exists_query_time = end_time_exists - start_time_exists

# Print the results
print(f"IN query time: {in_query_time} seconds")
print(f"EXISTS query time: {exists_query_time} seconds")

# Close the cursor and connection
cursor.close()
conn.close()


IN query time: 0.7569029331207275 seconds
EXISTS query time: 0.5043478012084961 seconds
